In [1]:
import sys
sys.path.append('/home/rida/GNN')

In [2]:
import Embedding_phase.Generate_data.Preprocess_attributes as pa
import Embedding_phase.Generate_data.config as args
from Embedding_phase.MGCN.Multi_GCN import MGCN 
from Search import search_motifs,save_motifs
import networkx as nx
import torch
import pickle

In [3]:
GST = nx.read_graphml(args.GST_DATA_PATH)

In [4]:
node_feature_map = pa.preprocess_node_features(GST)

In [5]:
GST = pa.Update_attributes(GST, node_feature_map)

All nodes updated with features.


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
input_dim = 18
hidden_dim = 64
out_dim = 16

In [8]:
model = MGCN(input_dim, hidden_dim,out_dim)

In [42]:
path = "/home/rida/GNN/Embedding_phase/models/best_mgcn_order_embedding_2000_pairs_5_k-hop_16.pth"  


In [43]:
model.load_state_dict(torch.load(path, map_location=device))  

/tmp/ipykernel_746919/2685531929.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device))


<All keys matched successfully>

In [44]:
model.eval()

MGCN(
  (pre_mp): Sequential(
    (0): Linear(in_features=18, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): ModuleList(
    (0-7): 8 x MGCNLayer(
      (convs): ModuleList(
        (0-4): 5 x GCNConv(64, 64)
      )
      (skip): Linear(in_features=64, out_features=64, bias=True)
      (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
  )
  (post_mp): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
  

In [ ]:
loaded_motifs = search_motifs(GST, model, node_feature_map, k_hop=12, num_pairs=500, s=2)


In [17]:
loaded_motifs

[(<networkx.classes.multigraph.MultiGraph at 0x7141f1543550>,
  2.8478501334916473e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f39eb3a0>,
  1.2426070838955638e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f39b35e0>,
  3.449608243499824e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f0f197b0>,
  1.2400881210794523e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f0873b20>,
  7.493709652806047e-09),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f08723b0>,
  9.009164969597805e-09),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f0873a90>,
  1.7977402677615828e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f0873df0>,
  1.5299509215083162e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f0871a20>,
  1.827814166688313e-08),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f0871a80>,
  1.8179741800850024e-07),
 (<networkx.classes.multigraph.MultiGraph at 0x7141f08710c0>,
  9.885150475952287e-09),
 (<networkx.classes.multig

In [18]:
type(loaded_motifs)

list

In [19]:
type(loaded_motifs[0])

tuple

In [20]:
terme_classe = { 0 :"Zones Urbanisees" ,
                 1 : "Zones de loisirs",
                 2 : "Zones agricoles",
                 3 : "Réseaux routier et ferroviaire et espaces associes",
                 4 : "Forets",
                 5 : "Espaces ouverts, sans ou avec peu de vegetation" ,
                 6 : "Eaux continentales",
                 7 : "Chantiers" }

In [21]:
RELATION_MAP = {
     0:"Adjacence",
     1:"Dérivation",
     2:"Continuation",
     3:"Scission",
     4:"Fusion"
}

In [22]:

len(loaded_motifs)

12

In [23]:
path = "/home/rida/GNN/Search_phase/Motifs_graphe/500_pairs_12_k-hop_32_N=4500_s=2.json"

In [24]:
import json

resultats = []

for i, (G, score) in enumerate(loaded_motifs, start=1):
    relations = []
    for u, v, d in G.edges(data=True):
        edge_score = d.get('edge_score')
        edge_type  = RELATION_MAP[int(d.get('edge_type'))]
        date_u = G.nodes[u].get('year')
        date_v = G.nodes[v].get('year')
        DN_u = G.nodes[u].get('DN')
        DN_v = G.nodes[v].get('DN')
        relations.append(f"{DN_u}/{date_u} --- {edge_type} ---> {DN_v}/{date_v} avec score = {edge_score}")

    header = {
        "motif_id": i,
        "score":    score,
        "size":     G.number_of_nodes(),
        "Relation": relations
    }

    entries = [header]
    for node_id, attrs in G.nodes(data=True):
        entry = {
            "Node":          str(attrs.get('DN')),
            "Date":          attrs.get('year'),
            "Classe_String": terme_classe[int(attrs.get('classe'))],
            "Classe_int":    int(attrs.get('classe')),
            "Aire":          attrs.get('Aire'),
            "Perimetre":     attrs.get('Perimeter'),
            "Largeur":       attrs.get('Largeur'),
            "Hauteur":       attrs.get('Hauteur'),
            "Rectangularity":attrs.get('Rectangularity'),
            "Elongation":    attrs.get('Elongation'),
            "I_Miller":      attrs.get('I_Miller'),
            "Mean":          attrs.get('Mean'),
            "std":           attrs.get('std'),
            "variance":      attrs.get('variance'),
            "centroid":      attrs.get('centroid'),
            "Nbr_Voisins":   attrs.get('Nbr_Voisins'),
            "surface_voisins": attrs.get('surface_voisins')
        }
        entries.append(entry)

    resultats.append(entries)


with open(path, "w", encoding="utf-8") as f:
    json.dump(resultats, f, ensure_ascii=False, indent=2)

print(f"✅ Fichier {path} généré avec succès.")


✅ Fichier /home/rida/GNN/Search_phase/Motifs_graphe/500_pairs_12_k-hop_32_N=4500_s=2.json généré avec succès.
